In [1]:
install.packages("readr")
install.packages("ggplot2")
install.packages("nortest") # ad.test
install.packages("agricolae") # kruskal with tukey groups
install.packages("rstatix") # dunn test, wilcox_test


The downloaded binary packages are in
	/var/folders/xk/kb8fhy7d7_j_wl1_hq_34ns40000gn/T//RtmpTyRqj0/downloaded_packages

The downloaded binary packages are in
	/var/folders/xk/kb8fhy7d7_j_wl1_hq_34ns40000gn/T//RtmpTyRqj0/downloaded_packages

The downloaded binary packages are in
	/var/folders/xk/kb8fhy7d7_j_wl1_hq_34ns40000gn/T//RtmpTyRqj0/downloaded_packages

The downloaded binary packages are in
	/var/folders/xk/kb8fhy7d7_j_wl1_hq_34ns40000gn/T//RtmpTyRqj0/downloaded_packages

The downloaded binary packages are in
	/var/folders/xk/kb8fhy7d7_j_wl1_hq_34ns40000gn/T//RtmpTyRqj0/downloaded_packages


In [1]:
#!/usr/bin/env Rscript
library(readr)
library(ggplot2)
library(nortest, pos=17) # ad.test
library(agricolae) # kruskal with tukey groups
library(rstatix) # dunn test, wilcox_test

source("config.r")


Attaching package: 'rstatix'


The following object is masked from 'package:stats':

    filter





The downloaded binary packages are in
	/var/folders/xk/kb8fhy7d7_j_wl1_hq_34ns40000gn/T//Rtmp6hoZXJ/downloaded_packages


In [2]:
give.n <- function(x){
  return(c(y = mean(x), label = length(x)))
}

In [3]:
subject="all"
target_file = "avg.csv"

results_loc <- paste("results", subject, target_file, sep="/")

approaches <- c("Ekstazi","Fastazi-S","FAST","Random")
v_factor_levels <- unique(approaches)

In [4]:
#==================================================
# Preparation
#==================================================
#LOAD RESULTS FILE
raw_results <- read_delim(results_loc, ",", escape_double = FALSE, trim_ws = TRUE)

raw_results <- subset(raw_results, Suite %in% approaches)

pdf_w <- 9
pdf_h <- 6

#reorder
raw_results$Suite <- factor(raw_results$Suite, levels=c("Ekstazi","FAST","Fastazi-S","Random"),
                            labels=c("Ekstazi","FAST","Fastazi","Random"))

Rows: 2705 Columns: 10

-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): Project, Suite
dbl (8): Version, Test count, APFDf, TTFF, pTTFF, Misses, Hit, HitCount


i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [5]:
#==================================================
# Boxplots (TTFF)
#==================================================
pdf(sprintf("%s.pdf", 'TTFF'), width=pdf_w, height=pdf_h/2)
# png("NAPFD.png",height=3,width=9)
ggplot(data=subset(raw_results, !is.na(pTTFF)), aes(Suite, pTTFF, fill=Suite)) +
  geom_violin(trim=TRUE) +
  geom_boxplot(width=0.1) +
  theme_minimal()  +
  theme(legend.position="none", axis.title = element_blank(), text = element_text(size = 22)) +
  theme(panel.grid.minor = element_blank()) +
  scale_fill_manual(values = my.cols)
dev.off()

pdf 
  2

In [6]:
#==================================================
# Boxplots (APFD)
#==================================================
pdf(sprintf("%s.pdf", 'NAPFD'), width=pdf_w, height=pdf_h/2)
ggplot(data=subset(raw_results, !is.na(APFDf)), aes(Suite, APFDf, fill=Suite)) +
  geom_violin(trim=TRUE) +
  geom_boxplot(width=0.1) +
  theme_minimal() +
  theme(legend.position="none", axis.title = element_blank(), text = element_text(size = 22)) +
  theme(panel.grid.minor = element_blank()) +
  scale_fill_brewer(palette="Set2") +
  scale_fill_manual(values = my.cols)
dev.off()

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.



pdf 
  2

In [34]:
#==================================================
# Normality Test
#==================================================
with(raw_results, ad.test(pTTFF))
with(raw_results, ad.test(APFDf))


	Anderson-Darling normality test

data:  pTTFF
A = 15.112, p-value < 2.2e-16



	Anderson-Darling normality test

data:  APFDf
A = 16.351, p-value < 2.2e-16


In [35]:
cat("\n################################################################################\n")
cat(  "#                                     TTFF                                     #")
cat("\n################################################################################\n")
#Kruskal-Wallis rank sum test
with(raw_results, tapply(pTTFF, Suite, median, na.rm=TRUE))
kruskal.test(pTTFF ~ Suite, data=raw_results)
cat("\n==================================================\n")

#Kruskal-Wallis rank sum test (MULTIPLE COMPARISON)
kruskalmc(raw_results$pTTFF, raw_results$Suite)
cat("\n==================================================\n")

cat("\n----------\n")
cat(  "# Pairwise comparisons using Wilcoxon’s test")
cat("\n----------\n")
wilcox_test(raw_results, pTTFF ~ Suite, p.adjust.method = "bonferroni")
cat("\n==================================================\n")

out <- kruskal(raw_results$pTTFF, raw_results$Suite)
out


################################################################################
#                                     TTFF                                     #
################################################################################


Ekstazi      FAST   Fastazi    Random 
0.3855574 0.4465524 0.3294574 0.4920391


	Kruskal-Wallis rank sum test

data:  pTTFF by Suite
Kruskal-Wallis chi-squared = 162.64, df = 3, p-value < 2.2e-16


Warning message in kruskalmc.default(raw_results$pTTFF, raw_results$Suite):
"20 lines including NA have been omitted"


,obs.dif,critical.dif,difference
,<dbl>,<dbl>,<lgl>
Ekstazi-FAST,278.35354,99.7666,TRUE
Ekstazi-Fastazi,30.31437,99.7666,FALSE
Ekstazi-Random,407.62687,99.7666,TRUE
FAST-Fastazi,248.03918,99.7666,TRUE
FAST-Random,129.27332,99.7666,TRUE
Fastazi-Random,377.31250,99.7666,TRUE




----------
# Pairwise comparisons using Wilcoxon<U+2019>s test
----------


,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>
1,pTTFF,Ekstazi,FAST,541,541,115341.0,2.34e-08,1.40e-07,****
2,pTTFF,Ekstazi,Fastazi,541,541,151023.0,1.46e-01,8.76e-01,ns
3,pTTFF,Ekstazi,Random,541,541,68596.5,1.31e-49,7.86e-49,****
4,pTTFF,FAST,Fastazi,541,541,174362.0,1.36e-09,8.16e-09,****
5,pTTFF,FAST,Random,541,541,137841.0,2.52e-01,1.00e+00,ns
6,pTTFF,Fastazi,Random,541,541,102002.0,2.09e-16,1.25e-15,****


$statistics
     Chisq Df p.chisq  t.value     MSD
  162.6432  3       0 1.961073 71.3389

$parameters
            test p.ajusted            name.t ntr alpha
  Kruskal-Wallis      none raw_results$Suite   4  0.05

$means
        raw_results.pTTFF      rank        std   r        Min       Max
Ekstazi         0.3613923  893.4263 0.13761084 536 0.01819172 0.6082222
FAST            0.4826452 1171.7799 0.30403945 536 0.00405954 1.0000000
Fastazi         0.3712187  923.7407 0.27535003 536 0.00270636 1.0000000
Random          0.4752075 1301.0532 0.09192415 536 0.05412541 0.6525840
              Q25       Q50       Q75
Ekstazi 0.2840148 0.3855574 0.4677709
FAST    0.2282127 0.4465524 0.7392244
Fastazi 0.1168594 0.3294574 0.5929617
Random  0.4393180 0.4920391 0.5346572

$comparison
NULL

$groups
        raw_results$pTTFF groups
Random          1301.0532      a
FAST            1171.7799      b
Fastazi          923.7407      c
Ekstazi          893.4263      c

attr(,"class")
[1] "group"

In [36]:
cat("\n################################################################################\n")
cat(  "#                                      NAPFD                                   #")
cat("\n################################################################################\n")
#Kruskal-Wallis rank sum test
with(raw_results, tapply(APFDf, Suite, median, na.rm=TRUE))
kruskal.test(APFDf ~ Suite, data=raw_results)
cat("\n==================================================\n")

#Kruskal-Wallis rank sum test (MULTIPLE COMPARISON)
kruskalmc(raw_results$APFDf, raw_results$Suite)
cat("\n==================================================\n")

cat("\n----------\n")
cat(  "# Pairwise comparisons using Wilcoxon’s test")
cat("\n----------\n")
wilcox_test(raw_results, APFDf ~ Suite, p.adjust.method = "bonferroni")
cat("\n==================================================\n")

out <- kruskal(raw_results$APFDf, raw_results$Suite)
out


################################################################################
#                                      NAPFD                                   #
################################################################################


Ekstazi      FAST   Fastazi    Random 
0.6193798 0.5526316 0.6724806 0.5129213


	Kruskal-Wallis rank sum test

data:  APFDf by Suite
Kruskal-Wallis chi-squared = 157.05, df = 3, p-value < 2.2e-16


,obs.dif,critical.dif,difference
,<dbl>,<dbl>,<lgl>
Ekstazi-FAST,276.55083,100.2306,TRUE
Ekstazi-Fastazi,31.60721,100.2306,FALSE
Ekstazi-Random,402.74399,100.2306,TRUE
FAST-Fastazi,244.94362,100.2306,TRUE
FAST-Random,126.19316,100.2306,TRUE
Fastazi-Random,371.13678,100.2306,TRUE




----------
# Pairwise comparisons using Wilcoxon<U+2019>s test
----------


,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>
1,APFDf,Ekstazi,FAST,541,541,174723.5,3.34e-08,2.00e-07,****
2,APFDf,Ekstazi,Fastazi,541,541,139087.5,1.58e-01,9.48e-01,ns
3,APFDf,Ekstazi,Random,541,541,221360.0,2.95e-48,1.77e-47,****
4,APFDf,FAST,Fastazi,541,541,115748.0,2.64e-09,1.58e-08,****
5,APFDf,FAST,Random,541,541,151851.5,2.84e-01,1.00e+00,ns
6,APFDf,Fastazi,Random,541,541,187545.0,1.08e-15,6.48e-15,****


$statistics
     Chisq Df p.chisq  t.value      MSD
  157.0488  3       0 1.961063 71.79725

$parameters
            test p.ajusted            name.t ntr alpha
  Kruskal-Wallis      none raw_results$Suite   4  0.05

$means
        raw_results.APFDf      rank       std   r Min       Max       Q25
Ekstazi         0.6384585 1260.2255 0.1500247 541   0 0.9834423 0.5359184
FAST            0.5183262  983.6747 0.3070212 541   0 0.9966171 0.2577519
Fastazi         0.6287228 1228.6183 0.2811242 541   0 0.9982877 0.4050388
Random          0.5256951  857.4815 0.1044111 541   0 0.9475248 0.4697856
              Q50       Q75
Ekstazi 0.6193798 0.7245614
FAST    0.5526316 0.7758621
Fastazi 0.6724806 0.8868613
Random  0.5129213 0.5663934

$comparison
NULL

$groups
        raw_results$APFDf groups
Ekstazi         1260.2255      a
Fastazi         1228.6183      a
FAST             983.6747      b
Random           857.4815      c

attr(,"class")
[1] "group"